In [2]:
!pip3 install langchain boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
from langchain.docstore.document import Document

In [3]:
example_doc_1 = """
Peter and Elizabeth took a taxi to attend the night party in the city. While in the party, Elizabeth collapsed and was rushed to the hospital.
Since she was diagnosed with a brain injury, the doctor told Peter to stay besides her until she gets well.
Therefore, Peter stayed with her at the hospital for 3 days without leaving.
"""

docs = [
    Document(
        page_content=example_doc_1,
    )
]

In [1]:
from typing import Dict

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.chains.question_answering import load_qa_chain
import json

query = """How long was Elizabeth hospitalized?
"""
context = """The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light, font sizes, line spacing, and more.

It comes with your choice of the Basic Pen or the Premium Pen, which you use to write on the screen like you would on paper. They also attach magnetically to your Kindle and never need to be charged. The Premium Pen includes a dedicated eraser and a customizable shortcut button.

The Kindle Scribe has the most storage options of all Kindle devices: choose from 8 GB, 16 GB, or 32 GB to suit your level of reading and writing.
"""
prompt_template = """Use the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
#prompts = ["""question: \"{question}"\\n\nContext: \"{context}"\\n\nAnswer:"""]

class ContentHandler(ContentHandlerBase):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({prompt: prompt, **model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

content_handler = ContentHandler()

chain = load_qa_chain(
    llm=SagemakerEndpoint(
        endpoint_name="jumpstart-example-huggingface-textgener-2023-04-09-15-28-58-828",
        #endpoint_name="huggingface-pytorch-inference-2023-04-09-15-10-22-749", 
        #credentials_profile_name="credentials-profile-name", 
        region_name="us-east-1", 
        model_kwargs={"temperature":1e-10},
        content_handler=content_handler
    ),
    prompt=PROMPT
)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

NameError: name 'docs' is not defined

In [11]:
print(PROMPT)

input_variables=['context', 'question'] output_parser=None partial_variables={} template='Use the following pieces of context to answer the question at the end.\n\n{context}\n\nQuestion: {question}\nAnswer:' template_format='f-string' validate_template=True


In [ ]:
content_type=application/json, input payload must be a json encoded dictionary with keys ['text_inputs', 'max_length', 'num_return_sequences', 'num_beams', 'top_p', 'early_stopping', 'do_sample', 'no_repeat_ngram_size', 'top_k', 'temperature', 'seed']